## rag chain

In [105]:
import os
os.environ['user_agent']='dd'

In [166]:
import os
import getpass

# Set your API keys and endpoint
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "farming-bot"

# It's a good practice to use getpass for sensitive information
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

In [144]:
import bs4
from langchain_ollama.llms import OllamaLLM
from bs4.filter import SoupStrainer
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### Steps
- Load Documents
- Split Doucments
- Embed Doucment
- Store vectors

In [145]:
# Load Documents
bs4_strainer = SoupStrainer("div",class_=("india-adda-Blog common-section","blogsContent"))
loader = WebBaseLoader("https://ibef.org/blogs/india-s-journey-towards-sustainable-low-carbon-farming",requests_per_second=3,
                       encoding="utf-8",bs_kwargs={"parse_only":bs4_strainer})
docs = loader.load()

In [146]:
docs

[Document(metadata={'source': 'https://ibef.org/blogs/india-s-journey-towards-sustainable-low-carbon-farming'}, page_content="\n\nIndia’s Journey Towards Sustainable, Low-Carbon Farming\n\n\n\n\n\n Sep 05, 2025, 23:55\n Agriculture\n IBEF\n\n\nAgriculture in India has always been more than just a sector—it is the soul of the nation’s economy. Feeding more than a billion people, providing livelihood for almost 46% of the workforce and contributing 16% (FY24) to the Gross Domestic Product (GDP), this industry has long been the backbone of India’s rural economy and national resilience. Today, this sector is rapidly emerging as a significant climate force, whether for better or worse.\nThe voluntary carbon market in India was estimated to be worth over Rs. 10,201 crores (US$ 1.2 billion), as of May 2023, with 1,451 projects either registered or undergoing various stages of consideration through two leading global registries, Verra and Gold Standard. India roughly comprises about 20% of the

In [147]:
#split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
all_splits=text_splitter.split_documents(docs)

In [148]:
all_splits

[Document(metadata={'source': 'https://ibef.org/blogs/india-s-journey-towards-sustainable-low-carbon-farming'}, page_content='India’s Journey Towards Sustainable, Low-Carbon Farming\n\n\n\n\n\n Sep 05, 2025, 23:55\n Agriculture\n IBEF'),
 Document(metadata={'source': 'https://ibef.org/blogs/india-s-journey-towards-sustainable-low-carbon-farming'}, page_content='Agriculture in India has always been more than just a sector—it is the soul of the nation’s economy. Feeding more than a billion people, providing livelihood for almost 46% of the workforce and contributing 16% (FY24) to the Gross Domestic Product (GDP), this industry has long been the backbone of India’s rural economy and national resilience. Today, this sector is rapidly emerging as a significant climate force, whether for better or worse.'),
 Document(metadata={'source': 'https://ibef.org/blogs/india-s-journey-towards-sustainable-low-carbon-farming'}, page_content='The voluntary carbon market in India was estimated to be wort

### all-MiniLM-L6-v2: This is a very popular choice due to its excellent balance of performance and efficiency. It is a small, fast model that generates 384-dimensional embeddings, making it ideal for applications with low latency requirements or limited resources

In [149]:
# Embed Chunks using huggingfaceEmbeddings
model="all-MiniLM-L6-v2"
model_kwargs={"device":"cpu"}
encode_kwargs={"normalize_embedding":True}
hf = HuggingFaceEmbeddings(model=model,
                           model_kwargs=model_kwargs,
                           encode_kwargs=encode_kwargs)
# create a vector store
vectorstore = Chroma.from_documents(documents=all_splits,embedding=hf)

## Generation & Retriever

In [150]:
retriever = vectorstore.as_retriever(search_type="similarity")

### Ollama model: 3.2:3b

In [167]:
llm = OllamaLLM(model="llama3.2:3b",
          temperature=0.7)

### fomrating doc function

In [152]:
def format_doc(doc):
    return "\n\n".join(doc.page_content for doc in docs)

## Response & Evaluation

In [153]:
from langchain.prompts import PromptTemplate

In [168]:
retriever.invoke("What does ‘low-carbon agriculture’ mean?")

[Document(metadata={'source': 'https://ibef.org/blogs/india-s-journey-towards-sustainable-low-carbon-farming'}, page_content='What does ‘low-carbon agriculture’ mean?\nLow-carbon agriculture actively minimises its contribution to global warming by reducing emissions. A low-carbon footprint for agriculture means low emissions of\u2002carbon dioxide (CO2) and other greenhouse gases due to all types of agricultural activities, such as fuel and other farm inputs, emissions from the field, livestock processes, etc. In broad terms, it means smart farming: implementing methods that minimise energy consumption and waste and improve the efficiency of every production stage.\nWhy it matters for India’s future?'),
 Document(metadata={'source': 'https://ibef.org/blogs/india-s-journey-towards-sustainable-low-carbon-farming'}, page_content='What does ‘low-carbon agriculture’ mean?\nLow-carbon agriculture actively minimises its contribution to global warming by reducing emissions. A low-carbon footpr

In [155]:
from langchain import hub
pull_prompt = hub.pull("rlm/rag-prompt")

In [156]:
# user query
query = "What does ‘low-carbon agriculture’ mean?"
# embeding query
embedding = hf.embed_query(query)

# define a  prompt
prompt_template = PromptTemplate.from_template(
    "Answer the question based only on the following context:\n{context}\n\nQuestion: {question}"
)
# Rag Pipeline
rag_chain=(
    {"context": retriever ,"question":RunnablePassthrough()}
           | prompt_template
           | llm
           | StrOutputParser()
           )
response = rag_chain.invoke(query)

response

"According to the context provided, 'low-carbon agriculture' means actively minimising its contribution to global warming by reducing emissions. It refers to a low-carbon footprint for agriculture, which means low emissions of carbon dioxide (CO2) and other greenhouse gases due to all types of agricultural activities. In essence, it means smart farming: implementing methods that minimise energy consumption and waste and improve the efficiency of every production stage."

In [157]:
response

"According to the context provided, 'low-carbon agriculture' means actively minimising its contribution to global warming by reducing emissions. It refers to a low-carbon footprint for agriculture, which means low emissions of carbon dioxide (CO2) and other greenhouse gases due to all types of agricultural activities. In essence, it means smart farming: implementing methods that minimise energy consumption and waste and improve the efficiency of every production stage."

Here is a simple flow chart:

```mermaid
graph TD;
    A[User Question] --> B(Retrieve Documents);
    B --> C{Context & Question};
    C --> D[Prompt Engineering];
    D --> E(LLM Inference);
    E --> F[Generated Answer];
```

### Tool Callings

In [158]:
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langchain_core.tools import tool

# 1. Define the tool with the @tool decorator
@tool
def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"It's  always sunnyin {city}!"

# 2. Instantiate the ChatOllama model
llm = ChatOllama(model="llama3.2:3b")

# 3. Create the agent with the chat model instance
# The prompt is handled automatically by create_react_agent
agent = create_react_agent(
    model=llm,
    tools=[get_weather]
)

# 4. Run the agent with a message
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in kolkata"}]})

# 5. Print the result
print(result)

{'messages': [HumanMessage(content='what is the weather in kolkata', additional_kwargs={}, response_metadata={}, id='b740ba55-b0bd-48ac-90b9-309b801d0381'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-09-18T16:30:10.2129802Z', 'done': True, 'done_reason': 'stop', 'total_duration': 479710800, 'load_duration': 101521800, 'prompt_eval_count': 160, 'prompt_eval_duration': 17304500, 'eval_count': 18, 'eval_duration': 360195300, 'model_name': 'llama3.2:3b'}, id='run--8916431a-98b9-4a70-94a7-677e3930507c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'kolkata'}, 'id': 'cd0b5b64-d612-49b9-9ea5-2ac47ca089b2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 160, 'output_tokens': 18, 'total_tokens': 178}), ToolMessage(content="It's  always sunnyin kolkata!", name='get_weather', id='0db77cc7-bcbf-4f3f-8d22-917cb0f65c38', tool_call_id='cd0b5b64-d612-49b9-9ea5-2ac47ca089b2'), AIMessage(content='The output from the tool c

## Indexing

In [159]:
import tiktoken
query = "What does ‘low-carbon agriculture’ mean??"
encoding_name="cl100k_base"
def num_token_from_string(question,encoding_name):
    """Return number of token in a text"""
    encoding = tiktoken.get_encoding(encoding_name=encoding_name)
    num_token = len(encoding.encode(question))
    return num_token

num_token_from_string(query,encoding_name)

10

In [160]:
all_splits
docs=[doc.page_content for doc in all_splits]
    

In [161]:
from langchain_huggingface import HuggingFaceEmbeddings

embd = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
query_result = embd.embed_query(query)
document_result = embd.embed_documents(docs)
print(len(query_result))

384


### Prompt Template

In [ ]:
from langchain import hub
pull_prompt = hub.pull("rlm/rag-prompt")
# Access the PromptTemplate object
prompt_template = pull_prompt.messages[0].prompt

# You can now use this object
print(prompt_template.template)

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## Cosine Similarity

In [164]:
import numpy as np

def cosine_similarity(vec1,vec2):
    vec2_flatten = np.array(vec2[0])
    dot_product = np.dot(vec1,vec2_flatten)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2_flatten)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result,document_result)
print(f"cosine similarity: {similarity}")

cosine similarity: 0.650755651651142
